# Preprocesamiento

### Li Yang, Patiy

## Ejercicios Parte II
Los ejercicios 5, 6, 7, 8 y 9 de esta parte deben leer su data de links en Github y el código y los resultados deben ser publicados en páginas en Github. Este es un proyecto distinto y los ejercicios tienen que ser publicados en una página diferente.

## Ejercicio 5

1. Get a polygons map of the lowest administrative unit possible.    
2. Get a table of variables for those units. At least 3 numerical variables.
3. Preprocess both tables and get them ready for merging. 
4. Do the merging, making the changes needed so that you keep the most columns.

**5.1. Subdivisiones administrativas a nivel de prefecturas en China (China_ADM2)**

Fuente: [China - Subnational Administrative Boundaries](https://data.humdata.org/dataset/cod-ab-chn) 

In [519]:
import geopandas as gpd

China_ADM2_Link = "https://github.com/Ciencia-de-datos-espaciales-2023-2/Evaluacion4_2da-Parte_Mining_GeoDFs/raw/main/maps/chn_adm_ocha_2020_shp/chn_admbnda_adm2_ocha_2020.shp"
China_ADM2 = gpd.read_file(China_prefectures_Link)
China_ADM2

ByeColumns = set(China_ADM2.columns)- {"ADM1_EN","ADM1_PCODE","ADM2_EN","ADM2_PCODE","geometry"}
China_ADM2.drop(columns = ByeColumns, inplace = True)
China_ADM2.rename(columns = {"ADM1_EN":"ADM1","ADM2_EN":"ADM2"}, inplace = True)

China_ADM2.ADM2_PCODE = China_ADM2.ADM2_PCODE.str.strip()
China_ADM2.ADM1_PCODE = China_ADM2.ADM1_PCODE.str.strip()

China_ADM2["ADM2"] =China_ADM2["ADM2"].str.replace(pat=r'\s|',repl="",regex=True)
China_ADM2["ADM1"] =China_ADM2["ADM1"].str.replace(pat=r'\s|',repl="",regex=True)

China_ADM2["ADM1_PROVINCE;ADM2_PREFECTURE"]=[";".join(pd) for pd in zip (China_ADM2.ADM1,China_ADM2.ADM2)]

China_ADM2 = China_ADM2.loc[:,["ADM1_PCODE","ADM2_PCODE","ADM1_PROVINCE;ADM2_PREFECTURE","geometry"]]
China_ADM2 = China_ADM2.to_crs(3415)

China_ADM2.to_file(os.path.join("maps","China3415_map","China3415_GeoMap.gpkg"), layer='PrefectureLevel_ADM2', driver="GPKG",index="")
China_ADM2

,ADM1_PCODE,ADM2_PCODE,ADM1_PROVINCE;ADM2_PREFECTURE,geometry
0,CN065,CN065100,XinjiangUygurAutonomousRegion;Karamay,"POLYGON ((-1970139.776 3501254.481, -1968482.7..."
1,CN065,CN065060,XinjiangUygurAutonomousRegion;ChangjiHuiAutono...,"POLYGON ((-1435866.505 3385323.591, -1441284.2..."
2,CN065,CN065130,XinjiangUygurAutonomousRegion;Shihezi,"POLYGON ((-1920371.401 3365547.617, -1922878.1..."
3,CN015,CN015120,InnerMongoliaAutonomousRegion;XilingolLeague,"POLYGON ((993809.799 3455055.691, 1001127.812 ..."
4,CN021,CN021020,LiaoningProvince;Benxi,"POLYGON ((1534098.705 2791209.973, 1528098.355..."
...,...,...,...,...
356,CN046,CN046110,HainanProvince;LingshuiLiAutonomousCounty,"MULTIPOLYGON (((86074.866 227768.818, 86071.62..."
357,CN046,CN046080,HainanProvince;Haikou,"MULTIPOLYGON (((141006.867 392722.649, 141006...."
358,CN046,CN046150,HainanProvince;Sanya,"MULTIPOLYGON (((15070.411 199741.112, 15069.53..."
359,CN041,CN041040,HenanProvince;Jiyuan,"POLYGON ((380643.001 2090118.562, 383534.651 2..."


**5.2. y 5.3. Preprocesamiento de tablas con variables numéricas**

No se logró conseguir una tabla con 3 variables numéricas, pero sí 3 tablas con 1 variable numérica cada una, por lo que se deberá realizar un *merging* de las 3 tablas. Las variables numéricas son las siguientes:
- `Índice normalizado de diferencia de vegetación` ( "normalized difference vegetation index" o NDVI). Fuente: [China: NDVI at Subnational Level](https://data.humdata.org/dataset/chn-ndvi-subnational)

- `Indicadores SDG 7.1.1 ("Sustainable Development Goals") del porcentaje de la población con acceso a la electricidad`. Fuente: [SDG Indicator 7.1.1: Access to Electricity, 2023 Release (2020 – 2022)](https://sedac.ciesin.columbia.edu/data/set/sdgi-7-1-1-access-electricity-2023)
- `Indicadores SDG 9.1.1 ("Sustainable Development Goals") del índice de acceso rural, que alude al porcentaje de la población que vive al menos a 2 kilómetros de una carretera transitable`. Fuente: [SDG Indicator 9.1.1: The Rural Access Index (RAI), 2023 Release (2015 – 2022)](https://sedac.ciesin.columbia.edu/data/set/sdgi-9-1-1-rai-2023)


`a) Preprocesamiento de tabla con índices normalizados de diferencia de vegetación (NDVI, por sus siglas en inglés) en subdivisiones administrativas a nivel de prefecturas (AMD2) en China`

Nota: Tabla contiene 334 datos

In [496]:
import pandas as pd

NDVI_Data = "https://github.com/Ciencia-de-datos-espaciales-2023-2/Evaluacion4_2da-Parte_Mining_GeoDFs/raw/main/data/chn-ndvi-adm2-5ytd.csv"
NDVI_China2019to2023 = pd.read_csv(ndvi_Data)
NDVI_China2023 = NDVI_China2019to2023[NDVI_China2019to2023.date=="2023-11-11"]
NDVI_China2023 = NDVI_China2023.reset_index()
NDVI_China2023.rename(columns = {"vim_avg": "NDVI_avg"}, inplace = True)

ByeColumns = set(NDVI_China2023.columns)- {"ADM2_PCODE","NDVI_avg"}
NDVI_China2023.drop(columns = ByeColumns, inplace = True)

NDVI_China2023.ADM2_PCODE =NDVI_China2023.ADM2_PCODE.str.strip()

NDVI_China2023.drop_duplicates(subset="ADM2_PCODE",keep="last", inplace=True)

import os
NDVI_China2023.to_csv(os.path.join("data","NDVI_China2023_prepro.csv"), index_label=False)
NDVI_China2023

,ADM2_PCODE,NDVI_avg
0,CN053160,0.6006
1,CN053020,0.6633
2,CN053060,0.6767
3,CN054050,0.3202
4,CN053130,0.6666
...,...,...
337,CN021040,0.3056
338,CN021120,0.2272
339,CN022090,0.3903
340,CN021070,0.238


`b) Preprocesamiento de tabla con indicadores del porcentaje de la población con acceso a la electricidad en subdivisiones administrativas a nivel de prefecturas (AMD2) en China`

Nota: Tabla contiene 344 datos

In [517]:
SDG711_ElectricityAccess_Data = "https://github.com/Ciencia-de-datos-espaciales-2023-2/Evaluacion4_2da-Parte_Mining_GeoDFs/raw/main/data/sdgi-7-1-1-access-electricity-2023-subnational.csv"

SDG711_ElectricityAccess_2023 = pd.read_csv(SDG711_ElectricityAccess_Data)
SDG711_ElectricityAccess_China2023 = SDG711_ElectricityAccess_2023[
    SDG711_ElectricityAccess_2023.ISO3=="CHN"]

SDG711_ElectricityAccess_China2023 = SDG711_ElectricityAccess_China2023.reset_index()
SDG711_ElectricityAccess_China2023.rename(columns = {
    "NAME_1":"ADM1","NAME_2":"ADM2","SDG711pct":"SDG711_pct"}, inplace=True)

SDG711_ElectricityAccess_China2023.ADM2 = SDG711_ElectricityAccess_China2023.ADM2.str.strip()
SDG711_ElectricityAccess_China2023.ADM1 = SDG711_ElectricityAccess_China2023.ADM1.str.strip()

SDG711_ElectricityAccess_China2023["ADM2"] = SDG711_ElectricityAccess_China2023[
    "ADM2"].str.replace(pat=r'\s|',repl="",regex=True)
SDG711_ElectricityAccess_China2023["ADM1"] = SDG711_ElectricityAccess_China2023[
    "ADM1"].str.replace(pat=r'\s|',repl="",regex=True)

SDG711_ElectricityAccess_China2023["ADM1_PROVINCE;ADM2_PREFECTURE"]=[";".join(pd) for pd in zip (
    SDG711_ElectricityAccess_China2023.ADM1,SDG711_ElectricityAccess_China2023.ADM2)]

ByeColumns = set(SDG711_ElectricityAccess_China2023.columns)- {"ADM1_PROVINCE;ADM2_PREFECTURE","SDG711_pct"}
SDG711_ElectricityAccess_China2023.drop(columns = ByeColumns, inplace = True)

SDG711_ElectricityAccess_China2023.to_csv(os.path.join(
    "data","SDG711_ElectricityAccess_China2023_prepro.csv"))

SDG711_ElectricityAccess_China2023

,SDG711_pct,ADM1_PROVINCE;ADM2_PREFECTURE
0,51.056611,Anhui;Anqing
1,70.505872,Anhui;Bengbu
2,57.124918,Anhui;Bozhou
3,65.594567,Anhui;Chaohu
4,50.803892,Anhui;Chizhou
...,...,...
339,79.097747,Zhejiang;Quzhou
340,94.671754,Zhejiang;Shaoxing
341,95.714004,Zhejiang;Taizhou
342,97.430320,Zhejiang;Wenzhou


` c) Preprocesamiento de tabla con indicadores del índice de acceso rural en subdivisiones administrativas a nivel de prefecturas (AMD2) en China`

Nota: Tabla contiene 344 datos

In [518]:
SDG911_RuralAccess_Data = "https://github.com/Ciencia-de-datos-espaciales-2023-2/Evaluacion4_2da-Parte_Mining_GeoDFs/raw/main/data/sdgi-9-1-1-Rural-Access-Index-rai-2023-subnational.csv"

SDG911_RuralAccess_2023 = pd.read_csv(SDG911_RuralAccess_Data)
SDG911_RuralAccess_China2023 = SDG911_RuralAccess_2023[SDG911_RuralAccess_2023.ISO3=="CHN"]

SDG911_RuralAccess_China2023 = SDG911_RuralAccess_China2023.reset_index()
SDG911_RuralAccess_China2023.rename(columns = {"NAME_1":"ADM1","NAME_2":"ADM2","SDG911pct":"SDG911_pct"}, inplace=True)

SDG911_RuralAccess_China2023.ADM2 = SDG911_RuralAccess_China2023.ADM2.str.strip()
SDG911_RuralAccess_China2023.ADM1 = SDG911_RuralAccess_China2023.ADM1.str.strip()

SDG911_RuralAccess_China2023["ADM2"] = SDG911_RuralAccess_China2023[
    "ADM2"].str.replace(pat=r'\s|',repl="",regex=True)
SDG911_RuralAccess_China2023["ADM1"] = SDG911_RuralAccess_China2023[
    "ADM1"].str.replace(pat=r'\s|',repl="",regex=True)

SDG911_RuralAccess_China2023["ADM1_PROVINCE;ADM2_PREFECTURE"]=[";".join(pd) for pd in zip (
   SDG911_RuralAccess_China2023.ADM1,SDG911_RuralAccess_China2023.ADM2)]

ByeColumns = set(SDG911_RuralAccess_China2023.columns)- {"ADM1_PROVINCE;ADM2_PREFECTURE","SDG911_pct"}
SDG911_RuralAccess_China2023.drop(columns = ByeColumns, inplace = True)

SDG911_RuralAccess_China2023.to_csv(os.path.join(
    "data","SDG911_RuralAccess_China2023_prepro.csv"))

SDG911_RuralAccess_China2023

,SDG911_pct,ADM1_PROVINCE;ADM2_PREFECTURE
0,84.749728,Anhui;Anqing
1,94.908604,Anhui;Bengbu
2,95.735585,Anhui;Bozhou
3,95.028170,Anhui;Chaohu
4,80.718768,Anhui;Chizhou
...,...,...
339,99.601174,Zhejiang;Quzhou
340,96.554436,Zhejiang;Shaoxing
341,94.300258,Zhejiang;Taizhou
342,92.875470,Zhejiang;Wenzhou


## Ejercicio 6

1. Choose a numeric variable from your merged data.
2. Decide which is the  best classification scheme for that variable. #Histograma para la variable numérica
3. Make a map for the best scheme. #Ejm: mapa de Perú para IDH
4. Make a histogram for the best scheme. #Con el mejor algoritmo

## Ejercicio 7

Compute the neighbors of the capital of your country. Plot the results for each of the options.

## Ejercicio 8

1. Compute the Moran's coefficient for **all** your numeric variables.
    
2. Make a scatter plot for each variable.

## Ejercicio 9
    
1. Compute the Local Moran for the variables in your data that have significant spatial correlation. 
2. Create a new column for each of those variables, with a label ('0 no_sig', '1 hotSpot', '2 coldOutlier', '3 coldSpot', '4 hotOutlier').
3. Prepare a map for each of the variables analyzed, showing the spots and outliers. (3 variables: 3 mapas)